# **1 数据读取**
这里承接的是之前的数据信息。我们读取的数据包含两部分：

1. **房屋特征**: 上一次回归过程中所利用的房屋特征（字段处理过后）；
    - *[yangpu_house.csv]*: 存储房屋特征的数据文件；

In [16]:
# 导入数据包
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
import re
from selenium import webdriver
import json
import re
warnings.filterwarnings('ignore')

# 工具1：展示df的基本属性
def show_df(df):
    return pd.DataFrame({
                'col' : df.columns.tolist(),
                'dtype' : [df[item].dtype for item in df.columns.tolist()],
                'uniqueVals' : [df[item].unique().shape[0] for item in df.columns.tolist()],
                'missingVals' : [df[item].isnull().sum() for item in df.columns.tolist()]
            }).sort_values(by='missingVals', ascending=False)

# 读取数据表

house = pd.read_csv('input/house_table.csv')
comm = pd.read_csv('input/comm_table.csv')
comm.head()

,comm_name,comm_area,comm_households,comm_plot_ratio,comm_parking_num,comm_green_ratio,comm_property_fee
0,翔顺公寓,暂无,NaN,1.8,NaN,25%,0.60元/m²
1,三门路510弄小区,58000m²,NaN,1.86,NaN,25%,0.84元/m²
2,腾越路465弄小区,暂无,NaN,暂无,NaN,暂无,0.60元/m²
3,国和二村,38400m²,NaN,2,NaN,40%,0.55元/m²
4,开鲁五村,36000m²,NaN,1.5,NaN,25%,0.55元/m²


# **2 地理信息扩充**

由于本身在优点、核心卖点、标题中对于地理位置信息的描述不是很清楚，这里决定在原有的特征基础上增加周边设施的特征，从两个方面考虑：

- 首先，是小区本身的地理信息特征，这是由小区的地址相联系的；
- 其次，是小区周围的配套生活设施，包含四类，分别是学校、商圈、地铁站、基础设施（医院、公园）；

通过selenium，对百度坐标拾取系统进行爬虫，我完成了这一步操作。

但是在此之前，先要确定我需要查询的内容：

---

## 备注1
我选择通过selenium完成这一步的爬虫，基于以下两个原因：
1. **意料之外的字符**：在我使用格式化输入来url （仅当含有'http://'）的时候，python似乎会自动在 '&' 符号后家上 'amp;'，而这一字符是无法 replace 掉的；
2. **编辑方式**：通过request，将这一链接进行post的时候，目前版本的百度API无法识别，会判为 “AK无效”，而selenium则可以不带 'http://' 完成url编辑(这样就不会被自动加上 'amp;'了)；

## 备注2
我选择通过[百度坐标拾取系统](http://api.map.baidu.com/lbsapi/getpoint/index.html)，而非[百度地图API（正/逆地理编码）](http://lbsyun.baidu.com/index.php?title=webapi/guide/webservice-geocoding)完成，基于以下原因：

1. **精度**：API的正/逆编码服务所提供的查询精度特别差，在我输入关键词比较模糊的时候，很难识别出我想要的指什么（都会返回默认的上层坐标，而不是小区或其他地点左边），而拾取系统似乎更加智能；


## **2.1 爬虫** 

In [2]:
primary_school = ['打虎山路第一小学', '水丰路小学', '复旦大学附属小学', '杨浦小学', '控江二村小学', '杨浦复旦科技园小学', '齐齐哈尔路第一小学', '杨浦六一小学', '上海市民办沪东外国语学校']

middle_school = ['兰生复旦中学', '上海同济大学附属存志学校', '复旦大学第二附属中学', '上海外国语大学附属双语学校', '上海控江中学附属民办学校']

high_school = ['复旦大学附属中学', '交通大学附属中学', '同济大学第一附属中学', '杨浦高级中学', '控江中学']

college = ['复旦大学', '上海财经大学', '同济大学', '上海体育学院']

busness_center = ['五角场', '鞍山新村', '黄兴公园', '江湾新城', '控江路', '平凉路', '中原路', '嫩江路']

subway = ['爱国路地铁站', '翔殷路地铁站', ' 市光路地铁站', '国权路地铁站', '鞍山新村地铁站', ' 三门路地铁站', '  江浦路地铁站', '黄兴路地铁站', ' 宁国路地铁站', '五角场地铁站', ' 杨树浦路地铁站', '江湾体育场地铁站', '四平路地铁站', '隆昌路地铁站', '大连路地铁站', '大柏树地铁站', '江湾镇地铁站']

hospital = ['上海交通大学医学院附属新华医院', '上海市肺科医院', '上海长海医院', '上海东方肝胆外科医院']

public_garden = ['和平公园', '黄兴公园', '杨浦公园', '新江湾城公园', '民星公园', '四平科技公园', '杨浦公园', '共青森林公园']

In [ ]:
chromedriver = 'C:/Users/Dushenghui/PycharmProjects/dailypy/venv/chromedriver.exe'
driver = webdriver.Chrome(executable_path=chromedriver)

base_url = 'http://api.map.baidu.com/lbsapi/getpoint/index.html'
driver.get(base_url)

In [ ]:
import time, random

def table_extloc(table_2q)
    global driver

    dic = {}
    for idx, item in enumerate(table_2q):
        print(item)
        driver.find_element_by_id('localvalue').clear()
        driver.find_element_by_id('localvalue').send_keys(item)
        driver.find_element_by_id('localsearch').click()
        time.sleep(random.random()*5)
        info = driver.find_element_by_xpath('//*[@id="no0"]').text
        print(info)
        dic[item] = info
        time.sleep(random.random()*5)

    df = pd.DataFrame({'loc':list(dic.keys()), 'info':list(dic.values())})
    return df

business_yp = table_extloc(busness_center)
garden_yp = table_extloc(public_garden)
hospital_yp = table_extloc(hospital)
subway_yp = table_extloc(subway)
school_yp = table_extloc(primary_school+middle_school+high_school+college)
comm_yp = table_extloc(comm.comm_name.unique)

# business_yp.to_csv('input/yangpu_business.csv', index=False)
# garden_yp.to_csv('input/yangpu_garden.csv', index=False)
# hospital_yp.to_csv('input/yangpu_hospital.csv', index=False)
# subway_yp.to_csv('input/yangpu_subway.csv', index=False)
# school_yp.to_csv('input/yangpu_school.csv', index=False)

## **2.2 数据读取** 
1. **房屋地理信息**: 根据房屋信息，回到百度地图中进行爬取的经纬度信息；
    - *[comm_geoloc.csv]*: 
2. **周边设施地理信息**: 存储杨浦区周边相关的配套设施信息，具体有：
    - *[yangpu_business.csv]*: 存储杨浦区<u>商圈信息</u>；
    - *[yangpu_garden.csv]*: 存储杨浦区<u>公园</u>信息；
    - *[yangpu_hospital.csv]*: 存储杨浦区<u>医院</u>信息；
    - *[yangpu_subway.csv]*: 存储杨浦区<u>地铁</u>信息；
    - *[yangpu_school.csv]*: 存储杨浦区<u>学校（众点小学、初中、高中、大学）</u>信息；

In [7]:
# business_yp = pd.read_csv('input/yangpu_business.csv')
# garden_yp = pd.read_csv('input/yangpu_garden.csv')
# hospital_yp = pd.read_csv('input/yangpu_hospital.csv')
# subway_yp = pd.read_csv('input/yangpu_subway.csv')
# school_yp = pd.read_csv('input/yangpu_school.csv')

business_yp['info']+=';'
garden_yp['info']+=';'
hospital_yp['info']+=';'
subway_yp['info']+=';'
school_yp['info']+=';'

business_yp.head()

,loc,info
0,五角场,五角场广场\n地址：上海市杨浦区中环路与黄兴路交叉路口南侧(UMAX悠迈生活广场北侧约50米...
1,鞍山新村,"鞍山新村\n途径地铁：地铁8号线\n坐标：121.516196,31.278969;"
2,黄兴公园,黄兴公园\n地址：上海市杨浦区营口路639号\n电话：(021)65512506\n坐标：1...
3,江湾新城,"江湾新城\n地址：上海市杨浦区\n坐标：121.517944,31.330478;"
4,控江路,"控江路\n地址：上海市杨浦区\n坐标：121.52174,31.279952;"


In [8]:
# comm_yp = pd.read_csv('input/comm_geoloc.csv')
comm_yp['info']+=';'
comm_yp.head()

,loc,info
0,文化佳园(公寓住宅),NaN
1,控江四村,"控江四村小区\n地址：上海市杨浦区松花江路977弄\n坐标：121.531555,31.29..."
2,市光一村,"市光一村\n地址：国和路80-87号\n坐标：121.542728,31.322039;"
3,鞍山八村,"鞍山八村住宅小区\n地址：上海市杨浦区彰武路\n坐标：121.517715,31.285928;"
4,本溪路165弄小区,本溪路165弄小区\n地址：上海市杨浦区本溪路165弄1~10号\n电话：(021)5596...


# **3 数据清洗**
可以看到这里的地理位置信息数据是一个字符串类型的网页文本，无法直接拿来进入模型分析，因此需要：

1. **检验错误**：
    - 在操作过程中，有些地理信息找到的匹配项是错误的，这是网站和网站之间的差异带来的，此时需要去结合代码和人工进行检验排查，并将正确的地理信息填入；
2. **数据填补**：
    - 会发现有些选项没有找到，这里的情况比较少（约三条），直接人力填补即可；

**3.1 小区地理坐标清洗**

本清洗流程分为两部分：

1. 首先填充缺失值；
2. 随后根据需要，通过正则匹配提取地理坐标；

In [9]:
def get_geo_loc_lon(loc_desc):
    import re
    try:
        lon = re.findall(r'坐标：(.*?),.*', loc_desc, re.S)[0]
        return lon
    except:
        return np.NaN

def get_geo_loc_lat(loc_desc):
    import re
    try:
        lat = re.findall(r'坐标：.*,(.*?);', loc_desc, re.S)[0]
        return lat
    except:
        return np.NaN

# 得到经纬度
def lat_lon(df):
    df['lat'] = df['info'].apply(get_geo_loc_lat).astype(np.float)
    df['lon'] = df['info'].apply(get_geo_loc_lon).astype(np.float)
    df = df.drop_duplicates(subset=['info'])
    return df[[x for x in df.columns if x not in ['info']]]

In [10]:
comm_yp[comm_yp['info'].isnull()]
loc_comm = {
    '文化佳园(公寓住宅)': '坐标：121.53171,31.294969;',
    '江南华府': '坐标：121.531555,31.290912;',
    '林园': '坐标：121.521332,31.266074;' 
}

comm_yp.loc[0, 'info'] = loc_comm['文化佳园(公寓住宅)']
comm_yp.loc[277, 'info'] = loc_comm['江南华府']
comm_yp.loc[304, 'info'] = loc_comm['林园']

# comm_yp.isnull().sum()
# loc     0
# info    0
# dtype: int64

In [11]:
comm_yp['lat'] = comm_yp['info'].apply(get_geo_loc_lat).astype(np.float)
comm_yp['lon'] = comm_yp['info'].apply(get_geo_loc_lon).astype(np.float)

del comm_yp['info']

# comm_yp.isnull().sum()
# loc     0
# info    0
# lon     0
# lat     0
# dtype: int64

comm_yp.iloc[0]

loc    文化佳园(公寓住宅)
lat        31.295
lon       121.532
Name: 0, dtype: object

In [12]:
comm_yp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 0 to 562
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   loc     563 non-null    object 
 1   lat     563 non-null    float64
 2   lon     563 non-null    float64
dtypes: float64(2), object(1)
memory usage: 13.3+ KB


**3.2 周边设施坐标清洗**

本清洗流程分为两部分：

1. 随后根据需要，通过正则匹配提取地理坐标；
2. 接着，根据地理坐标检验是否有重合项，并回到百度中重新检查(已从文档渠道完成数据更新)；

In [13]:
business_yp = lat_lon(business_yp)
# origin:((8, 4), unique:(8,))
garden_yp = lat_lon(garden_yp)
# origin((7, 4), unique(5,))
hospital_yp = lat_lon(hospital_yp)
# origin((4, 4), unique(4,))
subway_yp = lat_lon(subway_yp)
# origin((17, 4), unique(16,))
school_yp = lat_lon(school_yp)
# origin((28, 4), unique(24,))
business_yp.iloc[0]

loc        五角场
lat    31.3055
lon    121.522
Name: 0, dtype: object

# **4 特征构建**

这里会根据四个维度进行计算小区到周边生活设施的距离，并根据一定的范围，计算在这一半径下的特定设施数目，主要包含：

1. trans：小区附近的交通情况，这里用subway；
2. school：小区附近的学校情况，这里列入计算的是school；
3. business：小区周边的商圈，business会被列入计算；
4. fundamental：小区周边的基础设施，这里计算公园、医院；

但是在此之前，首先要去把原本DataFrame中的字段删除

In [18]:
fund_yp = pd.concat([hospital_yp, garden_yp]).reset_index(drop=True)

In [19]:
from geopy.distance import great_circle

# 舒适步行距离
comfort_distance = 1.5

def distance_count(lat, lon, df):
    tmpdf = df[[x for x in df.columns]]
    lst = []
    comm_tp = (lat, lon)
    for idx in list(tmpdf.index):
        item_tp = (tmpdf.loc[idx, 'lat'], tmpdf.loc[idx, 'lon'])
        lst.append(great_circle(item_tp, comm_tp).kilometers)

    count = (np.array(lst)<=comfort_distance).sum()
    return count

In [20]:
trans, school, business, fundamental = [], [], [], []

for idx_comm in list(comm_yp.index):
    comm_lat = comm_yp.loc[idx_comm, 'lat']
    comm_lon = comm_yp.loc[idx_comm, 'lon']
    trans.append(distance_count(lat=comm_lat, lon=comm_lon, df=subway_yp))
    school.append(distance_count(lat=comm_lat, lon=comm_lon, df=school_yp))
    business.append(distance_count(lat=comm_lat, lon=comm_lon, df=business_yp))
    fundamental.append(distance_count(lat=comm_lat, lon=comm_lon, df=fund_yp))

comm_yp['trans'] = trans
comm_yp['school'] = school
comm_yp['business'] = business
comm_yp['fundamental'] = fundamental

comm_yp



,loc,lat,lon,trans,school,business,fundamental
0,文化佳园(公寓住宅),31.294969,121.531710,3,7,2,2
1,控江四村,31.290912,121.531555,3,5,1,2
2,市光一村,31.322039,121.542728,2,2,2,3
3,鞍山八村,31.285928,121.517715,4,9,2,3
4,本溪路165弄小区,31.281318,121.521507,4,6,2,2
...,...,...,...,...,...,...,...
558,和平花苑,31.297570,121.530616,3,7,2,1
559,翡丽甲第,31.264367,121.525544,2,3,0,1
560,同济绿园,31.280502,121.518165,3,7,2,2
561,明丰阳光苑,31.321700,121.530939,2,3,1,3


In [22]:
comm_yp.to_csv('input/comm_geoloc.csv', index=False)